In [1]:
from __future__ import print_function
import torch

In [2]:
x = torch.Tensor(5, 3)
print(x)


 0.0000e+00  0.0000e+00  5.3883e+12
 4.5595e-41  2.0295e+18  4.5595e-41
 1.5290e-38  0.0000e+00  1.5290e-38
 0.0000e+00  2.6032e+19  4.5595e-41
 2.5144e+19  4.5595e-41  5.3913e+12
[torch.FloatTensor of size 5x3]



In [3]:
x = torch.rand(5, 3)
print(x)


 0.0699  0.3262  0.7917
 0.7570  0.6676  0.0684
 0.4864  0.4922  0.0418
 0.9692  0.5120  0.2818
 0.6988  0.1221  0.0789
[torch.FloatTensor of size 5x3]



In [6]:
print(x.size())

torch.Size([5, 3])


In [7]:
y = torch.rand(5,3)
z = torch.add(x,y)

In [8]:
print(z)


 0.4380  0.8311  1.0483
 0.8442  1.2819  0.1502
 0.7000  0.9350  0.1754
 1.2975  1.1765  0.7901
 0.7533  0.1624  0.3226
[torch.FloatTensor of size 5x3]



In [9]:
result = torch.rand(5,3)
torch.add(x,y, out=result)


 0.4380  0.8311  1.0483
 0.8442  1.2819  0.1502
 0.7000  0.9350  0.1754
 1.2975  1.1765  0.7901
 0.7533  0.1624  0.3226
[torch.FloatTensor of size 5x3]

In [10]:
#similar to numpy
b = torch.ones(5)
c = b.numpy()

In [11]:
print(c)

[ 1.  1.  1.  1.  1.]


**Convert ndArray to torchTensor**

In [16]:
import numpy as np
a = np.ones((5,6))
b = torch.from_numpy(a)
np.add(a,1,out = a)
print(a)
print(b)

[[ 2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.]
 [ 2.  2.  2.  2.  2.  2.]]

 2  2  2  2  2  2
 2  2  2  2  2  2
 2  2  2  2  2  2
 2  2  2  2  2  2
 2  2  2  2  2  2
[torch.DoubleTensor of size 5x6]



In [18]:
torch.cuda.is_available()#to check if cuda is available


False

**Gradients and Autograd**

In [19]:
from torch.autograd import Variable

In [22]:
x = Variable(torch.ones(2,2),requires_grad=True)
print(x)

Variable containing:
 1  1
 1  1
[torch.FloatTensor of size 2x2]



In [23]:
y = x + 2
print(y)

Variable containing:
 3  3
 3  3
[torch.FloatTensor of size 2x2]



In [24]:
print(y.grad_fn)

In [27]:
z = y**2*4
out = z.mean()
print(z, out)

Variable containing:
 36  36
 36  36
[torch.FloatTensor of size 2x2]
 Variable containing:
 36
[torch.FloatTensor of size 1]



In [28]:
out.backward()

In [31]:
print(x.grad)

Variable containing:
 6  6
 6  6
[torch.FloatTensor of size 2x2]



In [34]:
x = torch.randn(3)
x = Variable(x, requires_grad=True)
y = x * 2
while y.data.norm() < 1000:
        y = y * 2
print(y)

Variable containing:
  78.8790
 905.6913
-887.8644
[torch.FloatTensor of size 3]



In [35]:
gradients = torch.FloatTensor([0.1, 1.0, 0.0001])
y.backward(gradients)

print(x.grad)


Variable containing:
  102.4000
 1024.0000
    0.1024
[torch.FloatTensor of size 3]



****Neural Network****

In [36]:
import torch.nn as nn
import torch.nn.functional as F

In [41]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        #1 input channel, 6 output channel, 5*5 square conv
        self.conv2 = nn.Conv2d(6, 16, 5)
        #6 input channel, 16 output channel, 5*5 square conv
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        #Max pool over a (2*2) window
        x = F.max_pool2d(F.relu(self.conv1(x)),(2, 2))
        #if the size is a square you can only specify a number
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
print(net)

Net (
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear (400 -> 120)
  (fc2): Linear (120 -> 84)
  (fc3): Linear (84 -> 10)
)


In [42]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) #conv 1 weights

10
torch.Size([6, 1, 5, 5])


In [43]:
input = Variable(torch.randn(1, 1, 32, 32))
out = net(input)
print(out)

Variable containing:
 0.0168  0.1042 -0.0038 -0.0153  0.0506  0.0526 -0.0221  0.0541  0.0495 -0.0070
[torch.FloatTensor of size 1x10]



In [44]:
net.zero_grad()
out.backward(torch.randn(1, 10))

In [45]:
output = net(input)
target = Variable(torch.arange(1, 11))
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

Variable containing:
 38.2271
[torch.FloatTensor of size 1]



In [46]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU


In [47]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
Variable containing:
 0
 0
 0
 0
 0
 0
[torch.FloatTensor of size 6]

conv1.bias.grad after backward
Variable containing:
1.00000e-02 *
  1.8152
 -4.5390
 -8.6267
 -1.7651
 -1.7641
 -0.4112
[torch.FloatTensor of size 6]



In [48]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [49]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update
